### 日期和时间数据的类型及工具

In [1]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
now = datetime.now()
print(now)
print(now.year, now.month, now.day)     

2021-05-24 16:28:12.634761
2021 5 24


In [3]:
delta = datetime.now() - datetime(2018, 6, 24, 8, 15)
print(delta)
print(delta.days, delta.seconds)

1065 days, 8:13:12.649838
1065 29592


In [4]:
start = datetime(2021, 5, 24)
start + timedelta(19)

datetime.datetime(2021, 6, 12, 0, 0)

### 字符串与datetime互相转换
可以使用str方法或者传递一个指定的格式给strftime方法来对datetime对象和pandas的Timestamp对象进行格式化:

In [5]:
stamp = datetime(2021, 6, 12)
str_stamp = str(stamp)
strf_stamp = stamp.strftime('%Y-%m-%d')      # equivalents to stamp.strftime('%F')
weekday_stamp = stamp.strftime('%w')
print(str_stamp)
print(strf_stamp)
print(weekday_stamp)

2021-06-12 00:00:00
2021-06-12
6


### datetime格式
- %Y 四位数的年份
- %m 两位数的月份
- %d 两位数的日期
- %H 24小时制的小时
- %M 两位的分钟
- %S 秒
- %w 星期几，0是星期天
- %W 一年中的星期数，以星期一为每周的第一天，一年中第一个星期日前的日期为第0周
- %F %Y-%m-%d的简写  

可以使用datetime.strptime和这些格式代码，将字符串转换为日期，datetime.strptime是在已知格式的情况下转换日期的好方式：

In [6]:
value = '2021-06-12'
datetime.strptime(value, '%Y-%m-%d')

datetime.datetime(2021, 6, 12, 0, 0)

In [7]:
datestrs = ['6/12/2021', '6/12/2021']
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

[datetime.datetime(2021, 6, 12, 0, 0), datetime.datetime(2021, 6, 12, 0, 0)]

### pd.to_datetime

In [8]:
datestrs = ['2021-06-12 12:00:00', '2021-12-6 00:00:00']
pd.to_datetime(datestrs)

DatetimeIndex(['2021-06-12 12:00:00', '2021-12-06 00:00:00'], dtype='datetime64[ns]', freq=None)

to_dateframe方法还可以处理那些被认为是缺失值的值(None，空字符串等):

In [9]:
idx = pd.to_datetime(datestrs + [None])
idx

DatetimeIndex(['2021-06-12 12:00:00', '2021-12-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

In [10]:
idx[2]     # NaT，not a time是pandas中时间戳数据的是null值

NaT

In [11]:
pd.isnull(idx)

array([False, False,  True])

### 时间序列: 以时间戳为索引的Series或DateFrame

In [12]:
dates = [datetime(2021,4,1), datetime(2021,4,5), datetime(2021,4,7),
         datetime(2021,4,9), datetime(2021,4,10), datetime(2021,4,15)]
ts = Series(np.random.randn(6), index=dates)
ts

2021-04-01   -0.506298
2021-04-05   -1.212571
2021-04-07   -0.502677
2021-04-09   -1.805604
2021-04-10    0.048688
2021-04-15    1.780234
dtype: float64

In [13]:
ts.index

DatetimeIndex(['2021-04-01', '2021-04-05', '2021-04-07', '2021-04-09',
               '2021-04-10', '2021-04-15'],
              dtype='datetime64[ns]', freq=None)

In [14]:
# pandas使用Numpy的datetime64数据类型在纳秒级的分辨率下存储时间戳
ts.index.dtype

dtype('<M8[ns]')

不同索引的时间序列之间的算数运算在日期上自动对齐:

In [15]:
ts + ts[::2]        # ts[::2]将ts中每隔一个的元素取出

2021-04-01   -1.012597
2021-04-05         NaN
2021-04-07   -1.005354
2021-04-09         NaN
2021-04-10    0.097376
2021-04-15         NaN
dtype: float64

创建的时间序列Series的索引为DatetimeIndex对象，而DatetimeIndex对象的每个标量值是pandas的Timestamp对象，该对象可以保存频率信息:

In [16]:
ts[2]

-0.5026771472987005

### 索引、选择、子集
当基于标签进行索引和选择时，时间序列的行为和其他pandas.Series类似:

In [17]:
stamp = ts.index[2]
ts[stamp]

-0.5026771472987005

可以传入一个能解释为日期的字符串：

In [18]:
ts['20210401']

-0.5062984509650735

对于长时间序列，可以通过年、月获取时间序列的切片:

In [19]:
long_ts = Series(np.random.randn(1000), index=pd.date_range('1/1/2018', periods=1000))
long_ts

2018-01-01   -0.352418
2018-01-02   -0.734847
2018-01-03   -0.324235
2018-01-04   -1.335168
2018-01-05    1.339721
                ...   
2020-09-22   -1.471699
2020-09-23   -0.864000
2020-09-24   -0.985652
2020-09-25   -0.595978
2020-09-26    1.512416
Freq: D, Length: 1000, dtype: float64

In [20]:
long_ts['2018']

2018-01-01   -0.352418
2018-01-02   -0.734847
2018-01-03   -0.324235
2018-01-04   -1.335168
2018-01-05    1.339721
                ...   
2018-12-27    2.604432
2018-12-28    1.694335
2018-12-29    0.653961
2018-12-30    0.192852
2018-12-31   -0.640476
Freq: D, Length: 365, dtype: float64

In [21]:
long_ts['2018-12'].count()

31

In [22]:
long_ts['1/1/2020':'30/6/2020']

2020-01-01    1.346938
2020-01-02    0.430164
2020-01-03   -1.449413
2020-01-04   -1.982574
2020-01-05   -0.942428
                ...   
2020-06-26   -1.667307
2020-06-27   -0.332475
2020-06-28   -1.215730
2020-06-29   -0.525625
2020-06-30    0.036877
Freq: D, Length: 182, dtype: float64

注意！通过这种方式的切片产生了原时间序列的视图，意味着没有数据被复制，在切片上的修改会反映在原始数据上！

### 含有重复索引的时间序列

In [23]:
dates = pd.DatetimeIndex(['2021/4/14', '2021/5/23', '2021/5/23', '2021/6/13', '2021/6/13'])
dup_ts = Series(np.arange(5), index=dates)
dup_ts

2021-04-14    0
2021-05-23    1
2021-05-23    2
2021-06-13    3
2021-06-13    4
dtype: int32

In [24]:
dup_ts.index.is_unique

False

In [25]:
dup_ts['20210523']

2021-05-23    1
2021-05-23    2
dtype: int32

可通过groupby函数对其进行聚合:

In [26]:
grouped = dup_ts.groupby(level=0)
grouped.mean()

2021-04-14    0.0
2021-05-23    1.5
2021-06-13    3.5
dtype: float64

In [27]:
grouped.count()

2021-04-14    1
2021-05-23    2
2021-06-13    2
dtype: int64

### 日期范围、频率和移位 --> 生成日期范围
使用pd.date_range可以根据特定频率生成指定长度的DatetimeIndex:

In [28]:
index = pd.date_range('2021/6/9','2021/6/12')
index

DatetimeIndex(['2021-06-09', '2021-06-10', '2021-06-11', '2021-06-12'], dtype='datetime64[ns]', freq='D')

In [29]:
# 默认情况下产生的索引的间隔为天，通过freq参数可以设置其他频率
index = pd.date_range('2021/4/1','2021/6/1',freq='M')
index

DatetimeIndex(['2021-04-30', '2021-05-31'], dtype='datetime64[ns]', freq='M')

In [30]:
pd.date_range(start='2021/4/1', periods=4)

DatetimeIndex(['2021-04-01', '2021-04-02', '2021-04-03', '2021-04-04'], dtype='datetime64[ns]', freq='D')

In [31]:
pd.date_range(end='2021/6/1',periods=4)

DatetimeIndex(['2021-05-29', '2021-05-30', '2021-05-31', '2021-06-01'], dtype='datetime64[ns]', freq='D')

如果想生成标准化为零点的时间戳：

In [32]:
pd.date_range('2021-06-12 12:56:31', periods=4, normalize=True)

DatetimeIndex(['2021-06-12', '2021-06-13', '2021-06-14', '2021-06-15'], dtype='datetime64[ns]', freq='D')

### 日期范围、频率和移位 --> 频率和日期偏移
pandas中频率由基础频率和倍数组成，基础频率通常会有字符串别名，例如'M'代表月，'H'代表小时。对于每个基础频率，都有一个对象可以被用于定义日期偏移量:

In [33]:
from pandas.tseries.offsets import Hour, Minute

hour = Hour()
hour

<Hour>

可以传递一个整数来定义偏移量的倍数：

In [34]:
four_hours = Hour(4)
four_hours

<4 * Hours>

在大多数应用中，不需要显式地创建这些对象，而是使用字符串别名，如'4H'，在基础频率前放一个整数就可以生成倍数：

In [35]:
pd.date_range(start='2021/6/12', periods=4, freq='6H')

DatetimeIndex(['2021-06-12 00:00:00', '2021-06-12 06:00:00',
               '2021-06-12 12:00:00', '2021-06-12 18:00:00'],
              dtype='datetime64[ns]', freq='6H')

In [36]:
pd.date_range(start='2021/6/12', periods=4, freq='2H20min38s')      

DatetimeIndex(['2021-06-12 00:00:00', '2021-06-12 02:20:38',
               '2021-06-12 04:41:16', '2021-06-12 07:01:54'],
              dtype='datetime64[ns]', freq='8438S')

月中某星期的日期：

In [37]:
rnd = pd.date_range('2021-01-01', '2021-08-01', freq='WOM-3FRI')
rnd

DatetimeIndex(['2021-01-15', '2021-02-19', '2021-03-19', '2021-04-16',
               '2021-05-21', '2021-06-18', '2021-07-16'],
              dtype='datetime64[ns]', freq='WOM-3FRI')

Series和DataFrame都有一个shift方法用于进行简单的前向或后向移位，而不改变索引：

In [38]:
ts = Series(np.random.randn(4), index=pd.date_range('1/1/2021', periods=4, freq='M'))
ts

2021-01-31   -1.210239
2021-02-28    0.672817
2021-03-31   -0.849780
2021-04-30    0.619954
Freq: M, dtype: float64

In [39]:
ts.shift(2)         

2021-01-31         NaN
2021-02-28         NaN
2021-03-31   -1.210239
2021-04-30    0.672817
Freq: M, dtype: float64

In [40]:
ts.shift(-2)    

2021-01-31   -0.849780
2021-02-28    0.619954
2021-03-31         NaN
2021-04-30         NaN
Freq: M, dtype: float64

shift常用于计算时间序列或DataFrame多列时间序列的百分比变化：

In [41]:
ts / ts.shift(1) - 1

2021-01-31         NaN
2021-02-28   -1.555937
2021-03-31   -2.263019
2021-04-30   -1.729546
Freq: M, dtype: float64

由于简单移位并不改变索引，一些数据会被丢弃，因此如果频率是已知的，可以将频率传递给shift来推移时间戳而不是简单的数据:

In [42]:
ts.shift(2, freq='M')

2021-03-31   -1.210239
2021-04-30    0.672817
2021-05-31   -0.849780
2021-06-30    0.619954
Freq: M, dtype: float64

其他的频率也可以传递：    

In [43]:
ts.shift(3, freq='D')

2021-02-03   -1.210239
2021-03-03    0.672817
2021-04-03   -0.849780
2021-05-03    0.619954
dtype: float64

In [44]:
ts.shift(1, freq='90T')    # T表示分钟

2021-01-31 01:30:00   -1.210239
2021-02-28 01:30:00    0.672817
2021-03-31 01:30:00   -0.849780
2021-04-30 01:30:00    0.619954
dtype: float64

### 使用偏移进行移位日期
pandas的日期偏移也可以使用datetime或Timestamp对象完成：

In [45]:
from pandas.tseries.offsets import Day, MonthEnd

now = datetime.now()
now

datetime.datetime(2021, 5, 24, 16, 28, 13, 334892)

In [46]:
now + 3 * Day()

Timestamp('2021-05-27 16:28:13.334892')

In [47]:
now + MonthEnd()

Timestamp('2021-05-31 16:28:13.334892')

In [48]:
now + MonthEnd(2)

Timestamp('2021-06-30 16:28:13.334892')

锚定偏移可以用rollforward和rollback分别显式地将日期向前或向后移动:

In [49]:
offset = MonthEnd()
offset.rollforward(now)

Timestamp('2021-05-31 16:28:13.334892')

In [50]:
offset.rollback(now)

Timestamp('2021-04-30 16:28:13.334892')

将移位方法与groupby一起使用是日期偏移的一种创造性用法：

In [51]:
ts = Series(np.random.randn(20), index=pd.date_range('1/15/2021', periods=20, freq='4d'))
ts

2021-01-15    0.080126
2021-01-19    0.570829
2021-01-23   -0.403398
2021-01-27   -0.539619
2021-01-31   -0.658668
2021-02-04   -0.047140
2021-02-08   -0.969948
2021-02-12   -0.584366
2021-02-16   -0.906820
2021-02-20    0.318611
2021-02-24    0.442220
2021-02-28   -0.129912
2021-03-04   -0.254277
2021-03-08   -1.709824
2021-03-12   -0.586660
2021-03-16   -0.709958
2021-03-20   -0.827587
2021-03-24    0.880628
2021-03-28    1.367391
2021-04-01   -0.551236
Freq: 4D, dtype: float64

In [52]:
ts.groupby(offset.rollforward).mean()     # 月末

2021-01-31   -0.190146
2021-02-28   -0.268194
2021-03-31   -0.262898
2021-04-30   -0.551236
dtype: float64

使用resample是更快捷的方法：

In [53]:
ts.resample('M').mean()

2021-01-31   -0.190146
2021-02-28   -0.268194
2021-03-31   -0.262898
2021-04-30   -0.551236
Freq: M, dtype: float64

### 时期Period

In [54]:
p = pd.Period(2021, freq='A-DEC')    #A-DEC 每年指定月份的最后一个日历日
p

Period('2021', 'A-DEC')

In [55]:
p + 2

Period('2023', 'A-DEC')

In [56]:
p - 5

Period('2016', 'A-DEC')

In [57]:
pd.Period(2025, freq='A-DEC') - p

<4 * YearEnds: month=12>

In [58]:
date5 = pd.period_range('2021/6/1', '2021/12/5', freq='M')
pd.Series(np.arange(7), index=date5)

2021-06    0
2021-07    1
2021-08    2
2021-09    3
2021-10    4
2021-11    5
2021-12    6
Freq: M, dtype: int32

### 频率转换：Period和PeriodIndex对象可以通过asfreq方法转换频率

In [59]:
p = pd.Period(2021, freq='A-DEC')
p.asfreq('M', how='start')

Period('2021-01', 'M')

In [60]:
p.asfreq('M', how='end')

Period('2021-12', 'M')

In [61]:
date6 = pd.period_range('2018', '2021', freq='A-DEC')
date6

PeriodIndex(['2018', '2019', '2020', '2021'], dtype='period[A-DEC]', freq='A-DEC')

In [62]:
ps = pd.Series(np.arange(4), index=date6)
ps.asfreq('M',how='start')

2018-01    0
2019-01    1
2020-01    2
2021-01    3
Freq: M, dtype: int32

利用to_period方法可以将由时间戳索引的时间序列数据转换为以时期为索引

In [63]:
date7 = pd.date_range('2021/6/1', periods=4, freq='M')
s = pd.Series(np.arange(4), index=date7)
ps = s.to_period()
ps

2021-06    0
2021-07    1
2021-08    2
2021-09    3
Freq: M, dtype: int32

In [64]:
ps = s.to_period('A-DEC')     #指定转换的频率
ps

2021    0
2021    1
2021    2
2021    3
Freq: A-DEC, dtype: int32

In [65]:
ps = s.to_period()
ps

2021-06    0
2021-07    1
2021-08    2
2021-09    3
Freq: M, dtype: int32

In [66]:
ps.to_timestamp(how='start')      #逆操作

2021-06-01    0
2021-07-01    1
2021-08-01    2
2021-09-01    3
Freq: MS, dtype: int32

### 频率转换与重采样
重采样是时间序列频率转换的处理过程。高频率聚合到低频率称为降采样，反之为升采样:

In [67]:
date = pd.date_range(start='2021/4/1', periods=100, freq='D')
s = pd.Series(np.arange(100), index=date)
s.head()

2021-04-01    0
2021-04-02    1
2021-04-03    2
2021-04-04    3
2021-04-05    4
Freq: D, dtype: int32

In [68]:
s.resample('M').mean()          # 重采样，将以天为频率转换为以月为频率，聚合方法为平均值

2021-04-30    14.5
2021-05-31    45.0
2021-06-30    75.5
2021-07-31    95.0
Freq: M, dtype: float64

在降采样中重点考虑closed和label两个参数，其分别表示哪边区间是闭合的，哪边用于标记

In [69]:
date = pd.date_range(start='2021/4/1', periods=12, freq='D')
s = pd.Series(np.arange(12), index=date)
s.resample('5D', closed='right', label='right').sum()

2021-04-01     0
2021-04-06    15
2021-04-11    40
2021-04-16    11
Freq: 5D, dtype: int32

In [70]:
s.resample('5D', closed='left', label='left').sum()

2021-04-01    10
2021-04-06    35
2021-04-11    21
Freq: 5D, dtype: int32

在升采样中用到的不是聚合，而是需要对缺失值进行填充:

In [71]:
date = [datetime(2021,4,3), datetime(2021,4,13)]
s = pd.Series([2, 5], index=date)
s

2021-04-03    2
2021-04-13    5
dtype: int64

In [72]:
s.resample('D').ffill()

2021-04-03    2
2021-04-04    2
2021-04-05    2
2021-04-06    2
2021-04-07    2
2021-04-08    2
2021-04-09    2
2021-04-10    2
2021-04-11    2
2021-04-12    2
2021-04-13    5
Freq: D, dtype: int64

In [73]:
s.resample('D').ffill(2)     #设置填充的个数

2021-04-03    2.0
2021-04-04    2.0
2021-04-05    2.0
2021-04-06    NaN
2021-04-07    NaN
2021-04-08    NaN
2021-04-09    NaN
2021-04-10    NaN
2021-04-11    NaN
2021-04-12    NaN
2021-04-13    5.0
Freq: D, dtype: float64